In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pandas_profiling import ProfileReport

import matplotlib.pyplot as plt
from matplotlib import dates as md
import seaborn as sns
import plotly.graph_objs as go
import plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
cf.set_config_file(offline=True)

from sklearn.metrics import r2_score

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import lightgbm as lgb

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# How to add in elec data into regression model?
elec = pd.read_csv("../input/buildingdatagenomeproject2/electricity_cleaned.csv", index_col='timestamp', parse_dates=True)
elec = elec.loc[:, elec.columns.str.startswith('Robin_')]
elec

In [ ]:
df_meta = pd.read_csv('../input/buildingdatagenomeproject2/metadata.csv')
df_meta

In [ ]:
weather = pd.read_csv("../input/buildingdatagenomeproject2/weather.csv", index_col='timestamp', parse_dates=True)
weather

In [ ]:
weather_robin = weather[weather['site_id'].str.contains('Robin')]
weather_robin.head()

In [ ]:
elec['Robin_lodging_Elmer'].plot(figsize=(30,10))

In [ ]:
elec['Robin_education_So'].plot(figsize=(30,10))

In [ ]:
list_elecmeter = list(elec.columns)

for name_meter in list_elecmeter:
  df_daily = elec[name_meter].copy()
  df_daily = df_daily.resample('D').std() 

  list_abnormalDate = list(df_daily[df_daily==0].index.strftime('%Y-%m-%d'))
  elec['Date'] = pd.to_datetime(elec.index.date)

  elec.loc[elec['Date'].isin(list_abnormalDate), name_meter] = np.nan
  elec = elec.drop('Date', axis=1)

In [ ]:
#after cleaning abnormal days by checking std_daily==0 in previous code
elec['Robin_education_So'].plot(figsize=(30,10))

In [ ]:
name_meter = 'Robin_education_So'

In [ ]:
example_df = elec[[name_meter]].copy()

example_df['hour'] = example_df.index.hour
example_df['dayofweek'] = example_df.index.dayofweek

example_df = example_df.rename(columns={name_meter:'elec_meas'})
example_df = example_df[~example_df['elec_meas'].isna()]

traindata = example_df.loc[:'2016-12']
testdata = example_df.loc['2016-12':]

train_labels = traindata['elec_meas']

train_features = traindata.drop('elec_meas', axis=1)
test_features = testdata.drop('elec_meas', axis=1)

In [ ]:
LGB_model = lgb.LGBMRegressor()
LGB_model.fit(train_features, train_labels)

testdata['elec_pred'] = LGB_model.predict(test_features)

# Use the forest's predict method on the train data
example_df.loc[testdata.index, 'elec_pred'] = testdata['elec_pred']
testdata = testdata.drop('elec_pred', axis=1)

In [ ]:
mae = np.mean(abs(example_df['elec_pred'] - example_df['elec_meas']))
mae

In [ ]:
r2_score = r2_score(example_df.loc['2017', 'elec_meas'], example_df.loc['2017', 'elec_pred'])
r2_score

In [ ]:
alpha = 0.84

LGB_model_HI = lgb.LGBMRegressor(objective='quantile', alpha=alpha)
LGB_model_HI.fit(train_features, train_labels)

testdata['elec_pred_HI'] = LGB_model_HI.predict(test_features)

# Use the forest's predict method on the train data
example_df.loc[testdata.index, 'elec_pred_HI'] = testdata['elec_pred_HI']
testdata = testdata.drop('elec_pred_HI', axis=1)

LGB_model_LOW = lgb.LGBMRegressor(objective='quantile', alpha=1-alpha)
LGB_model_LOW.fit(train_features, train_labels)

testdata['elec_pred_LOW'] = LGB_model_LOW.predict(test_features)

# Use the forest's predict method on the train data
example_df.loc[testdata.index, 'elec_pred_LOW'] = testdata['elec_pred_LOW']
testdata = testdata.drop('elec_pred_LOW', axis=1)

example_df[['elec_meas', 'elec_pred', 'elec_pred_HI', 'elec_pred_LOW']].iplot()

In [ ]:
example_df = elec[[name_meter]].copy()

example_df['hour'] = example_df.index.hour
example_df['dayofweek'] = example_df.index.dayofweek

# add temperature
example_df = example_df.merge(weather_robin['airTemperature'], left_index=True, right_index=True)

example_df = example_df.rename(columns={name_meter:'elec_meas'})
example_df = example_df[~example_df['elec_meas'].isna()]


traindata = example_df.loc[:'2016-12']
testdata = example_df.loc['2016-12':]

train_labels = traindata['elec_meas']

train_features = traindata.drop('elec_meas', axis=1)
test_features = testdata.drop('elec_meas', axis=1)

In [ ]:
LGB_model = lgb.LGBMRegressor()
LGB_model.fit(train_features, train_labels)

testdata['elec_pred'] = LGB_model.predict(test_features)

# Use the forest's predict method on the train data
example_df.loc[testdata.index, 'elec_pred'] = testdata['elec_pred']
testdata = testdata.drop('elec_pred', axis=1)

In [ ]:
alpha = 0.84

LGB_model_HI = lgb.LGBMRegressor(objective='quantile', alpha=alpha)
LGB_model_HI.fit(train_features, train_labels)

testdata['elec_pred_HI'] = LGB_model_HI.predict(test_features)

# Use the forest's predict method on the train data
example_df.loc[testdata.index, 'elec_pred_HI'] = testdata['elec_pred_HI']
testdata = testdata.drop('elec_pred_HI', axis=1)

LGB_model_LOW = lgb.LGBMRegressor(objective='quantile', alpha=1-alpha)
LGB_model_LOW.fit(train_features, train_labels)

testdata['elec_pred_LOW'] = LGB_model_LOW.predict(test_features)

# Use the forest's predict method on the train data
example_df.loc[testdata.index, 'elec_pred_LOW'] = testdata['elec_pred_LOW']
testdata = testdata.drop('elec_pred_LOW', axis=1)

example_df[['elec_meas', 'elec_pred', 'elec_pred_HI', 'elec_pred_LOW']].iplot()

In [ ]:
example_df = elec[[name_meter]].copy()

example_df['hour'] = example_df.index.hour
example_df['dayofweek'] = example_df.index.dayofweek

# add temperature
example_df = example_df.merge(weather_robin['airTemperature'], left_index=True, right_index=True)

example_df = example_df.rename(columns={name_meter:'elec_meas'})
example_df = example_df[~example_df['elec_meas'].isna()]


traindata = example_df.loc[:'2016-12']
testdata = example_df.loc['2016-12':]

train_labels = traindata['elec_meas']

train_features = traindata.drop('elec_meas', axis=1)
test_features = testdata.drop('elec_meas', axis=1)

In [ ]:
example_df = elec.copy()

example_df['hour'] = example_df.index.hour
example_df['dayofweek'] = example_df.index.dayofweek

# add temperature
example_df = example_df.merge(weather_robin['airTemperature'], left_index=True, right_index=True)

example_df = example_df.rename(columns={name_meter:'elec_meas'})
example_df = example_df[~example_df['elec_meas'].isna()]


traindata = example_df.loc[:'2016-12']
testdata = example_df.loc['2016-12':]

train_labels = traindata['elec_meas']

train_features = traindata.drop('elec_meas', axis=1)
test_features = testdata.drop('elec_meas', axis=1)

In [ ]:
LGB_model = lgb.LGBMRegressor()
LGB_model.fit(train_features, train_labels)

testdata['elec_pred'] = LGB_model.predict(test_features)

# Use the forest's predict method on the train data
example_df.loc[testdata.index, 'elec_pred'] = testdata['elec_pred']
testdata = testdata.drop('elec_pred', axis=1)

In [ ]:
alpha = 0.84

LGB_model_HI = lgb.LGBMRegressor(objective='quantile', alpha=alpha)
LGB_model_HI.fit(train_features, train_labels)

testdata['elec_pred_HI'] = LGB_model_HI.predict(test_features)

# Use the forest's predict method on the train data
example_df.loc[testdata.index, 'elec_pred_HI'] = testdata['elec_pred_HI']
testdata = testdata.drop('elec_pred_HI', axis=1)

LGB_model_LOW = lgb.LGBMRegressor(objective='quantile', alpha=1-alpha)
LGB_model_LOW.fit(train_features, train_labels)

testdata['elec_pred_LOW'] = LGB_model_LOW.predict(test_features)

# Use the forest's predict method on the train data
example_df.loc[testdata.index, 'elec_pred_LOW'] = testdata['elec_pred_LOW']
testdata = testdata.drop('elec_pred_LOW', axis=1)

example_df[['elec_meas', 'elec_pred', 'elec_pred_HI', 'elec_pred_LOW']].iplot()

In [ ]:
alpha = 0.95

LGB_model_HI = lgb.LGBMRegressor(objective='quantile', alpha=alpha)
LGB_model_HI.fit(train_features, train_labels)

testdata['elec_pred_HI'] = LGB_model_HI.predict(test_features)

# Use the forest's predict method on the train data
example_df.loc[testdata.index, 'elec_pred_HI'] = testdata['elec_pred_HI']
testdata = testdata.drop('elec_pred_HI', axis=1)

LGB_model_LOW = lgb.LGBMRegressor(objective='quantile', alpha=1-alpha)
LGB_model_LOW.fit(train_features, train_labels)

testdata['elec_pred_LOW'] = LGB_model_LOW.predict(test_features)

# Use the forest's predict method on the train data
example_df.loc[testdata.index, 'elec_pred_LOW'] = testdata['elec_pred_LOW']
testdata = testdata.drop('elec_pred_LOW', axis=1)

example_df[['elec_meas', 'elec_pred', 'elec_pred_HI', 'elec_pred_LOW']].iplot()

In [ ]:
example_df['elec_pred_interval'] = example_df['elec_pred_HI'] - example_df['elec_pred_LOW']
example_df.pivot_table(index='hour', columns='dayofweek', values='elec_pred_interval').iplot()